# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import sqlite3


In [2]:
# Create the database school.sqlite 
conn = sqlite3.connect('school.sqlite')
cur = conn.cursor()


In [3]:
ls

contact_list.pickle  index.ipynb  README.md      school.sqlite-journal
CONTRIBUTING.md      LICENSE.md   school.sqlite


## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [4]:
# Your code here
cur.execute("""CREATE TABLE contactinfo ( 
                id INTEGER PRIMARY KEY, 
                firstName TEXT,
                lastName TEXT,
                role TEXT,
                tel TEXT,
                street TEXT,
                city TEXT,
                state TEXT,
                zipcode TEXT); """)

OperationalError: table contactinfo already exists

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [5]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [8]:
# # Iterate over the contact list and populate the contactInfo table here
# for contact in contacts:
#     firstName = contact['firstName']
#     lastName = contact['lastName']
#     role = contact['role']
#     tel  = contact['telephone ']
#     street = contact['street']
#     city = contact['city']
#     state = contact['state']
#     zipcode  = contact['zipcode ']
    
# cur.execute("""INSERT INTO contactInfo (firstName, lastName, role, tel, street, city, state, zipcode) 
#                   VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}');
#                 """.format(firstName, lastName, role, tel, street, city, state, zipcode) )

# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    firstName = contact['firstName']
    lastName = contact['lastName']
    role = contact['role']
    tel  = contact['telephone ']
    street = contact['street']
    city = contact['city']
    state = contact['state']
    zipcode  = contact['zipcode ']
    cur.execute("""INSERT INTO contactInfo (firstName, lastName, role, tel, street, city, state, zipcode) 
                  VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}');
                """.format(firstName, lastName, role, telephone, street, city, state, zipcode) )

**Query the Table to Ensure it is populated**

In [9]:
# Your code here 
cur.execute("""SELECT * 
               FROM contactInfo;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df



,id,firstName,lastName,role,tel,street,city,state,zipcode
0,1,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933
1,2,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
2,3,Christopher,Warren,student,2035687697,1935 University Hill Road,Champaign,IL,61938
3,4,Linda,Jacobson,staff,2035687697,479 Musgrave Street,Atlanta,GA,30303
4,5,Andrew,Stepp,student,2035687697,2981 Lamberts Branch Road,Hialeah,Fl,33012
5,6,Jane,Evans,student,2035687697,1461 Briarhill Lane,Abilene,TX,79602
6,7,Jane,Evans,student,2035687697,1461 Briarhill Lane,Abilene,TX,79602
7,8,Mary,Raines,student,2035687697,3975 Jerry Toth Drive,Ninilchik,AK,99639
8,9,Ed,Lyman,student,2035687697,3478 Be Sreet,Lansing,MI,48933


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [10]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [11]:
# Create the grades table
cur.execute('''CREATE TABLE grades(
   userId INTEGER NOT NULL,
   courseId INTEGER NOT NULL,
    grade INTEGER NOT NULL,
   PRIMARY KEY(userId,courseId)
);''')

OperationalError: table grades already exists

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [13]:
# Find the duplicate entry
cur.execute('''SELECT firstName, tel, count(*) FROM contactInfo
                GROUP BY firstName, lastName 
                HAVING count(*)> 1;''').fetchall()

# # Find the duplicate entry
# cur.execute("""SELECT firstName, lastName, tel, COUNT(*) 
#                FROM contactInfo
#                GROUP BY firstName, lastName, tel
#                HAVING COUNT(*) > 1;""").fetchall()

[('Ed', '5179695576', 2), ('Jane', '2035687697', 2)]

In [16]:
# Delete the duplicate entry
cur.execute('''DELETE FROM contactInfo WHERE tel = '5179695576' OR tel = '2035687697';''')

In [20]:
# Check that the duplicate entry was removed
cur.execute("""SELECT firstName, lastName, tel, COUNT(*) 
               FROM contactInfo
               GROUP BY firstName, lastName, tel
               HAVING COUNT(*) > 1;""").fetchall()

[]

## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [26]:
# Update Ed's address
cur.execute('''UPDATE contactInfo
                SET street = "2910 Simpson Avenue",
                city = "York",
                state = "PA",
                zipcode = "17403"
                WHERE firstName = "Ed" AND lastName = "Lyman";
''')

# # Update Ed's address
# cur.execute("""UPDATE contactInfo
#                SET street = "2910 Simpson Avenue",
#                    city = 'York',
#                    state = 'PA',
#                    zipcode = '17403'
#                WHERE firstName = "Ed" AND lastName = "Lyman";""")

In [27]:
# Query the database to ensure the change was made
cur.execute("""SELECT * 
               FROM contactInfo;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df


ValueError: Length mismatch: Expected axis has 0 elements, new values have 9 elements

## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [ ]:
# Your code here
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 